In [1]:
import numpy as np

In [3]:
n = 10
x = np.arange(n)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [9]:
def handle_chunk(chunk, nperseg=4, noverlap=1):
    step = nperseg - noverlap
    shape = x.shape[:-1] + ((x.shape[-1] - noverlap) // step, nperseg)
    strides = x.strides[:-1] + (step * x.strides[-1], x.strides[-1])
    result = np.lib.stride_tricks.as_strided(x, shape=shape, strides=strides)
    print(result)

In [10]:
handle_chunk(x)

[[0 1 2 3]
 [3 4 5 6]
 [6 7 8 9]]
